2. What’s Cooking?

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy as sp
from scipy import misc
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import sklearn as skl
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn import cross_validation
from sklearn import linear_model

a) Join the What’s Cooking competition on Kaggle.

b)Tell us about the data. How many samples (dishes) are there in the training set? How many categories (types of cuisine)? Use a list to keep all the unique ingredients appearing in the training set. How many unique ingredients are there?

from kaggle:
In the dataset, we include the recipe id, the type of cuisine, and the list of ingredients of each recipe (of variable length).

In [2]:
cooking_df_train = pd.read_json("train.json")
cooking_df_test = pd.read_json("test.json")

In [3]:
len(cooking_df_train)
cooking_df_train.iloc[39773]

cuisine                                                  mexican
id                                                          2362
ingredients    [green chile, jalapeno chilies, onions, ground...
Name: 39773, dtype: object

In [4]:
cooking_df_train.cuisine.unique()

array([u'greek', u'southern_us', u'filipino', u'indian', u'jamaican',
       u'spanish', u'italian', u'mexican', u'chinese', u'british', u'thai',
       u'vietnamese', u'cajun_creole', u'brazilian', u'french',
       u'japanese', u'irish', u'korean', u'moroccan', u'russian'], dtype=object)

In [5]:
len(cooking_df_train.cuisine.unique())

20

In [6]:
all_ingredients = [item for sublist in cooking_df_train.ingredients for item in sublist]

In [7]:
all_ingredients_unique = set(all_ingredients)
all_ingredients_unique = list(all_ingredients_unique)
len(all_ingredients_unique)
all_ingredients_unique[:4]

[u'low-sodium fat-free chicken broth',
 u'sweetened coconut',
 u'baking chocolate',
 u'egg roll wrappers']

(c) Represent each dish by a binary ingredient feature vector.
Use n × d feature matrix to represent all the dishes in training set and test set, where n is the number of dishes.

In [8]:
ingredient_fv = []
for ing in all_ingredients_unique:
    ingredient_fv.append(cooking_df_train.ingredients.apply(lambda x: 1 if ing in x else 0).values)
ingredient_fv = np.asarray(ingredient_fv)
ingredient_fv = ingredient_fv.T

In [11]:
print len(cooking_df_train.ingredients[0])
print all_ingredients_unique.index('grape tomatoes')
type(ingredient_fv)

9
5750


numpy.ndarray

(d) Using Naïve Bayes Classifier to perform 3 fold cross-validation on the training set and report your average classification accuracy. Try both Gaussian distribution prior assumption and Bernoulli distribution prior assumption.

In [55]:
#set y vector and X matrix
y = np.asarray(cooking_df_train.cuisine)
X = ingredient_fv

In [56]:
len(y)
X.shape

(39774, 6714)

In [28]:
#function for cross validation:
kf = cross_validation.KFold(39774, n_folds=3)
test_indices = []
train_indices = []
for train_index, test_index in kf:
    print("TRAIN:", len(train_index), "TEST:", len(test_index))
    train_indices.append(train_index)
    test_indices.append(test_index)

('TRAIN:', 26516, 'TEST:', 13258)
('TRAIN:', 26516, 'TEST:', 13258)
('TRAIN:', 26516, 'TEST:', 13258)


In [57]:
gnb = GaussianNB()

In [58]:
model = gnb.fit(X[train_indices[0]], y[train_indices[0]])
y_predict = model.predict(X[test_indices[0]])

In [67]:
count
float(count)/len(y_predict)

0.38105294916276966

In [61]:
model.score(X[test_indices[0]], y[test_indices[0]])

0.38105294916276966

In [68]:
predictions = []
scores = []
for i in xrange(0,3):
    model = gnb.fit(X[train_indices[i]], y[train_indices[i]])
    y_predict = model.predict(X[test_indices[i]])
    y_actual = y[test_indices[i]]
    predictions.append(y_predict)
    count = 0
    for i in xrange(len(y_predict)):
        if (y_predict[i] == y_actual[i]):
            count+=1
    scores.append(float(count)/len(y_predict))
                              

In [76]:
sum(scores)/float(len(scores))

0.3823854779504199

In [70]:
###bernoulli
bnb = BernoulliNB()

In [71]:
predictions_bnb = []
scores_bnb = []
for i in xrange(0,3):
    model = bnb.fit(X[train_indices[i]], y[train_indices[i]])
    y_predict = model.predict(X[test_indices[i]])
    y_actual = y[test_indices[i]]
    predictions_bnb.append(y_predict)
    count = 0
    for i in xrange(len(y_predict)):
        if (y_predict[i] == y_actual[i]):
            count+=1
    scores_bnb.append(float(count)/len(y_predict))

In [81]:
sum(scores_bnb)/float(len(scores_bnb))

0.6867048825866143

f) Using Logistic Regression Model to perform 3 fold cross-validation on the training set and report your average classification accuracy.

In [78]:
###logreg
predictions_log = []
scores_log = []
for i in xrange(0,3):
    logreg = linear_model.LogisticRegression()
    model = logreg.fit(X[train_indices[i]], y[train_indices[i]])
    y_predict = model.predict(X[test_indices[i]])
    y_actual = y[test_indices[i]]
    predictions_log.append(y_predict)
    count = 0
    for i in xrange(len(y_predict)):
        if (y_predict[i] == y_actual[i]):
            count+=1
    scores_log.append(float(count)/len(y_predict))

In [80]:
sum(scores_log)/float(len(scores_log))

0.7755568964650275

(g) Train your best-performed classifier with all of the training data, and generate test labels on test set. Submit your results to Kaggle and report the accuracy.

In [82]:
#X_test vector
X_test = []
for ing in all_ingredients_unique:
    X_test.append(cooking_df_test.ingredients.apply(lambda x: 1 if ing in x else 0).values)
X_test = np.asarray(X_test)
X_test = X_test.T

In [88]:
model = logreg.fit(X, y)
y_test = model.predict(X_test)

In [89]:
y_test

array([u'british', u'southern_us', u'italian', ..., u'italian',
       u'southern_us', u'mexican'], dtype=object)

In [95]:
submission = pd.DataFrame(cooking_df_test.id)
submission['cuisine'] = y_test

In [98]:
submission.to_csv("submission_cooking.csv",index=False)